In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
train_transforms = transforms.Compose([
                            transforms.Resize(115),
                            transforms.RandomCrop(115, fill = 0),
                            transforms.ToTensor()
                                      ])

In [4]:
valid_transforms = transforms.Compose([
                            transforms.Resize(115),
                            transforms.RandomCrop(115, fill = 0),
                            transforms.ToTensor()
                                      ])

In [5]:
train_data = datasets.ImageFolder(root = 'train',transform=train_transforms)

In [6]:
valid_data = datasets.ImageFolder(root = 'val',transform=valid_transforms)

In [7]:
fil1 = open('images_family_train.txt', 'r') 
Lin1 = fil1.readlines() 

In [8]:
c = list()

In [9]:
l = 0
temp = Lin1[0][8:]
for i in range(len(Lin1)):
    if(temp == Lin1[i][8:]):
        c.append(l)
    else:
        temp = Lin1[i][8:]
        l=l+1
        c.append(l)

In [10]:
fil2 = open('images_family_val.txt', 'r') 
Lin2 = fil2.readlines() 

In [11]:
d = list()

In [12]:
l = 0
temp = Lin2[0][8:]
for i in range(len(Lin2)):
    if(temp == Lin2[i][8:]):
        d.append(l)
    else:
        temp = Lin2[i][8:]
        l=l+1
        d.append(l)

In [13]:
for i in range(len(Lin1)):
    train_data.targets[i] = c[i]

In [14]:
for i in range(len(Lin2)):
    valid_data.targets[i] = d[i]

In [15]:
BATCH_SIZE = 8
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

In [16]:
class LeNet(nn.Module):
    def __init__(self, output_dim):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels = 3, 
                               out_channels = 6, 
                               kernel_size = 5)
        
        self.conv2 = nn.Conv2d(in_channels = 6, 
                               out_channels = 16, 
                               kernel_size = 5)
        
        self.fc_1 = nn.Linear(10000, 4000)
        self.fc_2 = nn.Linear(4000, 840)
        self.fc_3 = nn.Linear(840, output_dim)

    def forward(self, x):

        
        
        x = self.conv1(x)
        
        
        
        x = F.max_pool2d(x, kernel_size = 2)
        
        
        
        x = F.relu(x)
        
        x = self.conv2(x)
        
        
        x = F.max_pool2d(x, kernel_size = 2)
        
        
        
        x = F.relu(x)
        
        x = x.view(x.shape[0], -1)
        
        
        
        h = x
        
        x = self.fc_1(x)
        
        
        
        x = F.relu(x)

        x = self.fc_2(x)
    
        
        x = F.relu(x)

        x = self.fc_3(x)

        
        
        return x, h


In [17]:
OUTPUT_DIM = 100

model = LeNet(OUTPUT_DIM)

In [18]:
optimizer = optim.Adam(model.parameters())

In [19]:
criterion = nn.CrossEntropyLoss()

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
model = model.to(device)
criterion = criterion.to(device)

In [22]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [23]:
from tqdm import tqdm

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred, _ = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs



EPOCHS = 20

best_valid_loss = float('inf')

for epoch in tqdm(range(EPOCHS)):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


  5%|████                                                                            | 1/20 [03:41<1:09:59, 221.02s/it]

Epoch: 01 | Epoch Time: 3m 40s
	Train Loss: 4.612 | Train Acc: 0.63%
	 Val. Loss: 4.605 |  Val. Acc: 0.99%


 10%|████████                                                                        | 2/20 [07:08<1:05:06, 217.01s/it]

Epoch: 02 | Epoch Time: 3m 27s
	Train Loss: 4.608 | Train Acc: 0.69%
	 Val. Loss: 4.604 |  Val. Acc: 1.02%


 15%|████████████                                                                    | 3/20 [10:47<1:01:38, 217.56s/it]

...

Epoch: 08 | Epoch Time: 3m 35s
	Train Loss: 4.377 | Train Acc: 3.75%
	 Val. Loss: 4.547 |  Val. Acc: 2.88%


 45%|████████████████████████████████████▉                                             | 9/20 [32:30<39:42, 216.63s/it]

Epoch: 09 | Epoch Time: 3m 34s
	Train Loss: 4.286 | Train Acc: 5.01%
	 Val. Loss: 4.535 |  Val. Acc: 3.30%


 50%|████████████████████████████████████████▌                                        | 10/20 [36:04<35:59, 215.93s/it]

Epoch: 10 | Epoch Time: 3m 34s
	Train Loss: 4.168 | Train Acc: 6.06%
	 Val. Loss: 4.604 |  Val. Acc: 3.88%
